In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c kaggle-survey-2021
!unzip kaggle_survey_2021_responses.csv.zip

In [ ]:
!pip install --upgrade --user matplotlib

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

In [ ]:
response = pd.read_csv('kaggle_survey_2021_responses.csv')
response

In [ ]:
response.drop(index=0, inplace=True) # 첫째 행 삭제, column title은 추후 수정 예정
response.index = range(len(response)) # index 재조정
response = response.fillna('') # NaN → 빈값으로 변경
print(response.shape)
response.head()

In [ ]:
def total_answer(column):
  # 해당 컬럼명이 들어간 컬럼 추출
  col = response.columns[response.columns.str.contains(column)]
  tmp = response[col]

  # 컬럼별 응답 join
  total = tmp.apply(','.join, axis=1)

  # 정규표현식으로 필요없는 콤마 삭제
  import re
  ls = []
  for i in total:
    ls.append(re.sub(',{2,}', ',', i).strip(','))

  # 나눠진 응답 삭제
  response.drop(col, axis=1, inplace=True)
  # 통합 응답 추가
  response[column] = ls

In [ ]:
for i in range(7, 43):
  num = 'Q'+str(i) 
  total_answer(num) 

In [ ]:
response

In [ ]:
response.describe().T

## Q5 → 직군 나누기 (DA, DS, DE, OTHER)

In [ ]:
response['Q5'] = response['Q5'].replace(['Business Analyst', 'Data Analyst', 'Product Manager', 'Program/Project Manager', 'Statistician'],'DA')
response['Q5'] = response['Q5'].replace(['Data Engineer', 'DBA/Database Engineer', 'Machine Learning Engineer', 'Software Engineer', 'Developer Relations/Advocacy'], 'DE')
response['Q5'] = response['Q5'].replace(['Data Scientist', 'Research Scientist'], 'DS')
response['Q5'] = response['Q5'].replace(['Student', 'Currently not employed', 'Other'], 'OTHER')

In [ ]:
response['Q5'].value_counts()

* Q6 코딩, 프로그래밍 사용 연수 
* Q7 프로그래밍 사용 언어
* Q9 주 사용 IDE
* Q10 주 사용 노트북
* Q14 직군별 주 사용 시각화
* Q15 M/L 사용 년수
* Q20 현재 직업 군 or 최근 직업 군
* Q21 회사 규모
* Q25 연봉
* Q32, 33 빅데이터 관련 사용 제품 및 최다 빈도 제품


## Q6 코딩, 프로그래밍 사용 연수

In [ ]:
import itertools
plt.subplots(figsize=(20,15))
job = response['Q5'].unique()
length = len(job)
col_color = {'< 1 years':'r',
             'I have never written code':'#273c75',
             '1-3 years':'#273c75',
             '3-5 years':'#273c75',
             '5-10 years':'#273c75',
             '10-20 years':'#273c75',
             '20+ years':'#273c75'}

for i, j in itertools.zip_longest(job, range(length)):
  plt.subplot((length/2), 2, j+1)
  data = response[response['Q5'] == i]['Q6'].value_counts(ascending=True)
  data.plot(kind='barh', color=pd.DataFrame(data).reset_index()['index'].replace(col_color))
  plt.title(i, size=20)

## Q7 프로그래밍 사용 언어
* Python 
* R
* SQL 
* C
* C++
* Java
* Javascript 
* Julia
* Swift
* Bash
* MATLAB 
* None
* Other

In [ ]:
response['Q7'] = response['Q7'].str.replace('C\++', '++').str.replace('Javascript', 'JS')

In [ ]:
lang = ['Python', 'R', 'SQL', 'C', '\++', 'Java', 'JS', 'Julia', 'Swift', 'Bash', 'MATLAB', 'None', 'Other']

In [ ]:
plt.subplots(figsize=(20,17))
job = response['Q5'].unique()
length = len(job)

for i, j in itertools.zip_longest(job, range(length)):
  plt.subplot((length/2), 2, j+1)
  data = response[response['Q5'] == i]  
  
  lan = []
  cnt = []
  for k in lang:
    lan.append(k)
    cnt.append(len(data[data['Q7'].str.contains(k)]))
  
  df = pd.DataFrame(columns=["language", 'cnt'])
  df['language'] = lan
  df['cnt'] = cnt
  df = df.sort_values(by='cnt', ascending=False)
  sns.barplot(data=df, y='language', x='cnt')
  plt.title(i, size=20)
  plt.xlabel('')
  plt.ylabel('')